<img src='images/logo_politecnico.png'>
<label>Alberto Benincasa s251415</label>

In [12]:
"""
Import delle librerie necessarie per lo svolgimento della tesina
"""
import pandas as pd
import numpy as np
import itertools
import time
import seaborn as sns

from collections import defaultdict, Counter
from scipy import interp
from prettytable import PrettyTable
from functools import wraps

"""
MATPLOTLIB
"""
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams

"""
WARNINGS
"""
import warnings
warnings.filterwarnings("ignore")
import math
from scipy.cluster import hierarchy as hc
import scipy.spatial as scs
import scipy.stats as ss

"""
SKLEARN
"""
from sklearn.metrics import log_loss, precision_score, recall_score, confusion_matrix, roc_curve, accuracy_score, roc_auc_score, f1_score, auc
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold, GridSearchCV, learning_curve, cross_val_score
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline, Pipeline

"""
PLOTLY
"""
import plotly
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
chart_studio.tools.set_credentials_file(username='albertobenincasa', api_key='14CCI1wSA0JGaDZhuouL')

In [8]:
"""
Definizione delle costanti che verranno utilizzate nel codice
"""

PLOTLY_COLORS = ['#4A708B', '#87CEFA']
PLOTLY_OPACITY = 1
COLORSCALE_HEATMAP = [         [0.0, '#011f4b'],
                [0.1111111111111111, '#03396c'], 
                [0.2222222222222222, '#005b96'], 
                [0.3333333333333333, '#2171b5'], 
                [0.4444444444444444, '#6497b1'], 
                [0.5555555555555556, '#6baed6'], 
                [0.6666666666666666, '#B0E2FF'], 
                [0.7777777777777778, '#b3cde0'], 
                [0.8888888888888888, '#bdd7e7'], 
                               [1.0, '#BFEFFF']] 
COLOR_PALETTE = sns.color_palette("Blues").as_hex()
RANDOM_SEED = 11

"""
Definizione dei parametri che verranno utilizzati in fase di classificazione
"""

LOGISTIC_REGRESSION_PARAMS = [{
    'clf__solver': ['liblinear'],
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'clf__penalty': ['l2', 'l1']
},
{
    'clf__solver': ['newton-cg', 'lbfgs'], 
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l2']
}]
SVM_PARAMS = [
{
    'clf__kernel': ['linear'],
    'clf__C': [0.1, 1, 10, 100],
}, 
{
    'clf__kernel': ['rbf'],
    'clf__C': [0.1, 1, 10, 100],
    'clf__gamma': [0.1, 1, 10, 100],
}]

RANDOM_FOREST_PARAMS = {
    'clf__max_depth': [50, 75, 100],
    'clf__max_features': ["sqrt", "log2"],
    'clf__criterion': ['gini', 'entropy'],
    'clf__n_estimators': [100, 300, 500]
}

KNN_PARAMS = {
    'clf__n_neighbors': [2, 3, 5, 10, 15],
    'clf__weights': ['uniform', 'distance'],
    'clf__p': [1, 2, 10]
}

### INTRODUZIONE

Il Machine Learning è una branca dell'analisi dati che ha l'obbiettivo di utilizzare questi ultimi per costruire un algoritmo in grado di predirre il comportamento di quelli nuovi. 
Le tecniche di machine learning sono state una vera rivoluzione nel data mining, in grado di avere un forte impatto sui più svariati campi di applicazione (es: Netflix per i film da consigliare agli utenti, Amazon per consigliare l'acquisto di specifici prodotti agli acquirenti, i diversi assistenti vocali oggi esistenti e persino per la costruzione di macchine a guida autonoma).

L'obbiettivo di questa tesina è quello di analizzare un insieme di dati reperibili online per poterne fare, innanzitutto, un processo di pre-analisi, e successivamente confrontare diversi modelli predittivi in modo tale da verificare quale funzioni meglio, sottolineando comunque che i risultati variano molto in base al tipo di dati di cui si è in possesso.

Il dataset oggetto di questa analisi, visualizzabile e scaricabile all'indirizzo https://www.kaggle.com/ronitf/heart-disease-uci, consta di una serie di parametri di pazienti affetti o no da disturbi di tipo cardiaco; esso deriva da un database contenente in origine 76 attributi, ma tutti le analisi effettuate si riferiscono ad un sottoinsieme di essi di 14.

Con *disturbi di tipo cardiaco* utilizziamo un termine generico per indicare un problema che causa il non corretto funzionamento del cuore. I tre principali e più comuni disturbi sono:
* **Coronaropatia**: una qualsiasi alterazione, anatomica o funzionale, delle arterie coronarie, cioè dei vasi sanguigni che portano sangue al muscolo cardiaco.
* **Insufficienza cardiaca**:  incapacità del cuore di fornire il sangue in quantità adeguata rispetto all'effettiva richiesta dell'organismo.
* **Aritmia**: condizione clinica nella quale viene a mancare la normale frequenza o la regolarità del ritmo cardiaco.

Scendendo più nel dettaglio del nostro dataset, apprendiamo che questi 14 attributi specificano diverse caratteristice, di seguito elencate:
* **age**: età del paziente in anni
* **sex**: sesso del paziente (1=maschio, 0=femmina)
* **cp**: tipologia di dolore addominale riscontrata (1=Angina pectoris tipica, 2=Angina pectoris atipica, 3=Dolore non dovuto ad Angina pectoris , 4=Asintomatico)
* **trestbps**: pressione del sangue a riposo del paziente (in mmHg all'ammissione in ospedale)
* **chol**: colesterolo del paziente misurato in mg/dl
* **fbs**: glicemia a digiuno > 120 mg/dl (1=vero; 0=falso) 
* **restecg**: risutati elettrocardiografici a riposo (0=normale, 1=presenza di anormalità nell'onda ST-T, 2=probabile ipertrofia ventricolo sinistro)  
* **thalach**: battito cardiaco massimo raggiunto
* **exang**: angina pectoris indotta da esercizio (1=si, 0=no) 
* **oldpeak**: sottoslivellamento del tratto ST indotta dall'esercizio, non presente a riposo
* **slope**: la pendenza durante il picco di esercizio del tratto ST
* **ca**: numero di vasi maggiori (0-3) colorati per fluoroscopia
* **thal**: malattia del sangue chiamata talassemia (3 = normale; 6 = difetto fisso; 7 = difetto reversibile)
* **target**: presenza di malattia (1=si, 0=no)

### CARICAMENTO DEL DATASET E ANALISI PRELIMINARI

Come prima cosa carichiamo il dataset in una variabile di tipo DataFrame utilizzando la libreria <code>pandas</code>.

In [9]:
# Carico il dataset presente in locale
dataset = pd.read_csv("./dataset/heart.csv")

Il passo successivo consiste nel guardare il dataset appena caricato in modo da capirne il contenuto, le diverse feature disponibili e il tipo di dato presente in quest'ultime.

In [15]:
# Dimensione del dataset
print("Il dataset ha %d record e %d features.\n" % dataset.shape)

# Conteggio del numero di classi per la classificazione
print(f"Ci sono {dataset['target'].unique().size} classi differenti:"f"\n {dataset['target'].unique().tolist()}")

# Conteggio del numero di valori unici per ogni colonna
print(f"\nValori unici per ogni campo: \n{dataset.nunique()}")

Il dataset ha 303 record e 14 features.

Ci sono 2 classi differenti:
 [1, 0]

Valori unici per ogni campo: 
age          41
sex           2
cp            4
trestbps     49
chol        152
fbs           2
restecg       3
thalach      91
exang         2
oldpeak      40
slope         3
ca            5
thal          4
target        2
dtype: int64


### PREPROCESSING

Prima di poter passare alla classificazione, è fondamentale una fase di preprocessing dei dati. 
Questo è lo step più importante di tutti, perchè pulire i dati prima di inviarli in input ai diversi algoritmi di classificazione ha un duplice vantaggio: per prima cosa, evitiamo di sovraccaricare la potenza di calcolo della macchina per effettuare operazioni che poi con tutta probabilità risulteranno ininfluenti per la classificazione finale, e per seconda, analizzare i dati prima di processarli ci permette di ottenere la classificazione migliore possibile.

Inoltre, è molto importante anche controllare la presenza di eventuali valori nulli: nel caso in cui ci fossero colonne non valorizzate, è bene procedere con l'eliminazione di quest'ultime per ridurre il numero di dati su cui dobbiamo lavorare.

La strategia alternativa all'eliminazione di intere colonne, particolarmente vantaggiosa solo se tutti i valori (o perlomeno, la maggior parte) sono non definiti è quella di inserire valori di default (come 0) per i record che non hanno valori definiti.

Come possiamo notare, non vi è traccia della presenza di valori nulli nel dataset in analisi.

In [16]:
# Controllo la presenza di valori nulli
print(f"\nConteggio dei valori NaN: \n{dataset.isnull().sum()}")


Conteggio dei valori NaN: 
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


La prima, semplice, analisi da fare è quella di vedere la tipologia dei dati presenti.

In [17]:
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Come possiamo notare, tutti i nostri attributi sono di tipo numerico. Nonostante questo, dalla descrizione del dataset sappiamo che alcuni di questi valori corrispondono in realtà ad elementi di tipo categorico. 

Infatti, i campi valorizzati con delle stringhe non permettono un'analisi semplice ed immediata, [in quanto sarebbe corretto encodare PARLARE DI QUESTO]

Vediamo ora come i nostri campioni sono suddivisi nelle diverse classi

In [18]:
y = dataset['target'].value_counts()
print(y)

1    165
0    138
Name: target, dtype: int64


Fortunatamente, il dataset è abbastanza bilanciato: abbiamo circa lo stesso numero di campioni classificati come malati e non malati. Questo è molto importante perchè possiamo assegnare lo stesso peso alle due classi quando passeremo alla fase di classificazione.

Inoltre, possiamo notare come il nostro compito di classificazione sarà di tipo binario: infatti, il nostro obiettivo è riuscire a predire se un paziente potrebbe riscontrare malattie cardiache oppure no. Possiamo ora approfondire alcuni dettagli statistici sul dataset utilizzando il metodo <code>describe()</code> sul nostro DataFrame di pandas. L'output mostrerà:
* **count** $\to$ specifica il numero dei record presenti nel dataset 
* **mean** $\to$ specifica la media dell'attributo calcolata per tutti i record
* **std** $\to$ specifica la deviazione standard dell'attributo
* **min** $\to$ specifica il valore minimo dell'attributo
* **25%** $\to$ il 25% dei record ha un valore minore di quello visualizzato (lower percentile)
* **50%** $\to$ il 50% dei record ha un valore minore di quello visualizzato (median percentile)
* **75%** $\to$ il 75% dei record ha un valore minore di quello visualizzato (upper percentile)
* **max** $\to$ specifica il valore massimo dell'attributo

In [19]:
dataset.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [20]:
def create_bar(type, data, col, visible=False):
    """
    Creazione di un istogramma   
    """    
    
    if type == "sick":
        c = PLOTLY_COLORS[0]
    else:
        c = PLOTLY_COLORS[1]
    
    return go.Histogram(
        x = data[col],
        name = type,
        marker = dict(color = c),
        visible = visible,
        opacity = PLOTLY_OPACITY,
    )

def feature_histogram(data, feature, title):
    """
    Stampa l'istogramma della distribuzione del valore per una funzione
    
    :param (DataFrame) data: il dataset
    :param (string) feature: attributo che vogliamo visualizzare
    :param (string) title: titolo del grafico
    """
    
    trace1 = create_bar("sick", data[data['target'] == 1], feature, True)
    trace2 = create_bar("healthy", data[data['target'] == 0], feature, True)
    
    data = [trace1, trace2]
    
    layout = dict(
        title=title,
        autosize=True,
        yaxis=dict(
            title='value',
            automargin=True,
        ),
        legend=dict(
            x=0,
            y=1,
        ),
        barmode='group',
        bargap=0.15,
        bargroupgap=0.1
    )
    fig = dict(data=data, layout=layout)

    return py.iplot(fig, filename=title)

feature_histogram(dataset, 'target', 'Distribuzione classi')

A questo punto possiamo analizzare la distribuzione dei nostri dati usando dei boxplot: un boxplot è un modo standardizzato per visualizzare la distribuzione dei dati sulla base di un riepilogo di cinque numeri (min, lower percentile, median, upper percentile, max); risulta molto utile per visualizzare dati anomali e per controllare se i dati sono ditribuiti in modo simmetrico, quanto sono aggregati.

Le informazioni che possiamo ricavare da un box plot sono:
* **median**(Q2 / 50esimo percentile) $\to$ il valore medio del dataset
* **first quartile**(Q1 / 25esimo percentile) $\to$ il valore medio tra il valore più piccolo (non il minimo) e il median del dataset
* **third quartile**(Q3 / 75esimo percentile) $\to$ il valore medio tra il valore più grande (non il massimo) e il median del dataset
* **interquartile range**(IQR) $\to$ dal 25esimo al 75esimo percentile
* **outliers** $\to$ elementi visualizzati come singoli punti
* **maximum** $\to$ Q3 + 1.5xIQR
* **minimum** $\to$ Q1 - 1.5xIQR

Chiaramente, ha poco senso visualizzare in questo tipo di grafici quegli attributi che sono binari o che presentano poche opzioni, per cui filtreremo questi dati prima di visualizzare i grafici.

In [21]:
def create_box(type, data, col, visible=False):
    """
    Creazione di una box
    """   
    if type == 'sick':
        c = PLOTLY_COLORS[0]
    else:
        c = PLOTLY_COLORS[1]
        
    return go.Box(
        y = data[col],
        name = type,
        marker = dict(color = c),
        visible = visible,
        opacity = PLOTLY_OPACITY,
    )

sicks = dataset[dataset['target'] == 1]
healthy = dataset[dataset['target'] == 0]
box_features = [col for col in dataset.columns if ((col != 'class') and (dataset[col].nunique() > 5))]

active_index = 0

box_sick = [(create_box('sick', sicks, col, False) if i != active_index
           else create_box('sick', sicks, col, True))
             for i, col in enumerate(box_features)
            ]
box_healthy = [(create_box('healthy', healthy, col, False) if i != active_index
            else create_box('healthy', healthy, col, True))
             for i, col in enumerate(box_features)
            ]

data = box_sick + box_healthy
number_of_features = len(box_features)
steps = []

for i in range(number_of_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * number_of_features],
        label = box_features[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(
    active = active_index,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    autosize=True,
    yaxis=dict(
        title='valori',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='box_slider')

commenti sul grafico sopra

Gli Istogrammi sono un tipo di grafico molto utile alla rappresentazione di variabili di tipo categorico che si presenta con delle barre rettangolari la cui altezza è proporzionale ai valori che rappresentano.
Con uno slider possiamo muoverci lungo le diverse features, per visualizzare meglio la distribuzione dei diversi valori.

In [22]:
hist_features = [col for col in dataset.columns if (col != 'target')]

active_index = 0
hist_sick = [(create_bar('sick', sicks, col, False) if i != active_index
           else create_bar('sick', sicks, col, True))
             for i, col in enumerate(hist_features)
            ]
hist_healthy = [(create_bar('healthy', healthy, col, False) if i != active_index
           else create_bar('healthy', healthy, col, True))
             for i, col in enumerate(hist_features)
            ]

total_data = hist_sick + hist_healthy
number_of_features = len(hist_features)
steps = []

for i in range(number_of_features):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * number_of_features],
        label = hist_features[i],
    )
    step['args'][1][i] = True
    steps.append(step)
    
sliders = [dict(
    active = active_index,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    autosize=True,
    yaxis=dict(
        title='valori',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=total_data, layout=layout)
py.iplot(fig, filename='bar_slider')

Come possiamo notare dal grafico soprastante, valori come *age*, *trestbs* e *chol* non sono troppo determinanti per la classificazione. Altri attributi come *cp*, *thal*, *slope* e *thalach* hanno valori tutto sommato ben distinti tra le due classi: questi saranno gli attributi che saranno più importanti nella fase di classificazione.

Una matrice di correlazione è una tabella che mostra i coefficienti di correlazione tra insiemi di variabili. 
Ogni variabile casuale ($X_i$) nella tabella è correlata con ciascuno degli altri valori nella tabella ($X_j$); questo permette di vedere quali coppie hanno la più alta correlazione.
La correlazione si riferisce a qualsiasi associazione statistica, ma nell'uso comune del termine si indica quanto due variabili siano vicine ad avere una relazione lineare l'una con l'altra.

In questo frangente useremo la correlazione di Pearson, che è una misura della correlazione lineare tra due variabili $X$ e $Y$. Secondo la diseguaglianza di Cauchy-Schwartz, tale valore è compreso tra +1 e -1, dove +1 è la correlazione positiva totale, 0 è nessuna correlazione lineare e -1 è una correlazione negativa totale.

Questo coefficiente è calcolato come $ \frac{cov(X, Y)}{\sigma_X\sigma_Y} $ dove:
* $cov$ si chiama covarianza ed è calcolata come ($\mu$ è il valore medio)  $$ cov(X, Y) = E[(X - \mu_X)(Y - \mu_Y)] $$
* $\sigma_X$ è la deviazione standard di $X$ $$ \sigma_X = \sqrt{E[X^2] - (E[X])^2} $$
* $\sigma_Y$ è la deviazione standard di $Y$ $$ \sigma_Y = \sqrt{E[Y^2] - (E[Y])^2} $$

In [23]:
def plot_correlation_matrix(matrix, title):

    z_text = np.around(matrix.values.tolist(), decimals=2)

    figure = ff.create_annotated_heatmap(z=matrix.values, 
                                         x=matrix.columns.tolist(), 
                                         y=matrix.index.tolist(),
                                         annotation_text=z_text,
                                         colorscale=COLORSCALE_HEATMAP,
                                         showscale=True)

    figure.layout.title = title
    figure.layout.autosize = False
    figure.layout.width = 850
    figure.layout.height = 850
    figure.layout.margin = go.layout.Margin(l=140, r=100, b=200, t=80)
    figure.layout.xaxis.update(side='bottom')
    figure.layout.yaxis.update(side='left')

    for i in range(len(figure.layout.annotations)):
        figure.layout.annotations[i].font.size = 8
                                    
    return py.iplot(figure, filename=title)

def plot_correlation_row(matrix, title):
    
    matrix = pd.Series.to_frame(matrix.loc['class']).transpose()
    z_text = np.around(matrix.values.tolist(), decimals=2)

    figure = ff.create_annotated_heatmap(z=matrix.values, 
                                         x=matrix.columns.tolist(), 
                                         y=matrix.index.tolist(),
                                         annotation_text=z_text,
                                         colorscale=COLORSCALE_HEATMAP,
                                         showscale=False)

    figure.layout.title = title
    figure.layout.autosize = False
    figure.layout.width = 850
    figure.layout.height = 220
    figure.layout.xaxis.update(side='bottom')
    figure.layout.yaxis.update(side='left')

    for i in range(len(figure.layout.annotations)):
        figure.layout.annotations[i].font.size = 8


    return py.iplot(figure, filename=title)


In [24]:
corr_dataset = dataset.corr(method='pearson')
plot_correlation_matrix(corr_dataset, 'Matrice di correlazione')

Guardando ai valori della matrice di correlazione riportata sopra, possiamo notare facilmente che gli attributi presenti nel nostro dataset non sono molto correlati.

Ciò implica che dobbiamo includere tutte le features, in quanto possiamo e conviene eliminare solo quelle in cui la correlazione di due o più features è molto elevata.

Considerando però l'attributo *target*, che ricordiamo specifica se il paziente in analisi è affetto oppure no da qualche disturbo cardiaco, notiamo che i fattori principali che ne discriminano il valore sono le features *cp (chest pain)*, *thalach* e *slope*; ciò l'avevamo ipotizzato anche guardando l'istogramma, per cui ciò ne conferma ulteriormente l'importanza.

Un dendrogramma è un diagramma che rappresenta un albero. Questa rappresentazione schematica viene utilizzata in contesti diversi, ma vedremo il caso in cui si rappresenta il clustering gerarchico.
Essa illustra la disposizione dei cluster e l'obiettivo è quello di analizzare se abbiamo caratteristiche duplicate. Questo rientra sempre nell'analisi in vista di una possibile riduzione della dimensionalità dei dati.

Il criterio di collegamento determina la distanza tra insiemi di osservazioni come una funzione della distanza a coppie tra osservazioni (The linkage criterion determines the distance between sets of observations as a function of the pairwise distances between observations). Useremo UPGMA(Unweighted Pair Group Method With Arithmetic Mean), per cui la prossimità tra due cluster viene calcolata come la media aritmetica .. (Proximity between two clusters is the arithmetic mean of all the proximities between the objects of one, on one side, and the objects of the other, on the other side. The method is frequently set the default one in hierarhical clustering packages.)

In [25]:
names = dataset.columns
inverse_correlation = 1 - corr_dataset # This is the 'dissimilarity' method

fig = ff.create_dendrogram(inverse_correlation, 
                           labels=names, 
                           colorscale=COLOR_PALETTE,
                           linkagefun=lambda x: hc.linkage(x, 'average'))

fig['layout'].update(dict(
    title="Dendrogramma di correlazione tra gli attributi",
    width=800, 
    height=600,
    xaxis=dict(
        title='Features',
    ),
    yaxis=dict(
        title='Distance',
    ),
))
py.iplot(fig, filename='dendrogram_corr_clustering')

Come vediamo dal grafico sopra, non abbiamo distanze molto piccoli tra i clusters, e ciò, insieme al precedente grafico della matrice di correlazione, ci rafforza l'idea che tenere tutti gli attributi in fase di classificazione è la strada giusta da prendere.

### NORMALIZZAZIONE DEI DATI

La maggior parte delle volte, i set di dati contengono caratteristiche molto variabili in termini di grandezze, unità di misura e range. 
Poichè la maggior parte degli algoritmi di machine learning utilizza la distanza euclidea per misurare la distanza tra due dati nei loro calcoli, questo potrebbe causare dei problemi.

Se lasciati così come sono, questi algoritmi prendono solo la grandezza delle funzioni che trascurano le unità: i risultati variano molto tra le diverse unità, ad esempio tra 5Kg e 5000g.

Per questo, le caratteristiche con grandezze elevate peseranno molto di più nei calcoli rispetto agli attributi con minor grandezza. Per risolvere questo problema, dobbiamo portare tutti gli attributi sullo stesso livello di grandezza e ciò si può fare con il ridimensionamento.

Per fare ciò, useremo lo StandardScaler, che standardizza i nostri dati sia con la media che con la deviazione standard. L'operazione matematica eseguita sarà: $$ x' = \frac{x - \mu_x}{\sigma_x} $$
Dopodichè, spezzeremo il nostro dataset in un array di dati non classificati e un array di label, che utilizzeremo nella fase di classificazione.

In [26]:
def dataframe_to_array(data):
    X_data = data.drop(['target'], axis=1)
    y_data = data['target']
    return X_data, y_data

def scale_data(X_data):
    scaler = StandardScaler(with_mean=True, with_std=True, copy=True)
    return scaler.fit_transform(X_data)

X_data, y_data = dataframe_to_array(dataset)
X_scaled_data = scale_data(X_data)

### PRINCIPAL COMPONENT ANALYSIS

Quando i nostri dati sono rappresentati da una matrice troppo grande (ossia il numero di dimensioni è troppo alto) risulta difficile estrarre le caratteristiche più interessanti e trovare correlazioni tra di loro, senza considerare che lo spazio occupato è più alto del necessario.
La PCA è una tecnica che consente di ridurre la dimensionalità dei dati preservando le differenze più importanti che interccorrono tra i campioni.

Questa trasformazione è definita in modo tale che la prima componente principale abbia la varianza più grande possibile (ovvero, tiene conto il più possibile della variabilità dei valori assunti da una variabile) e ciascun componente successivo ha a sua volta la varianza più alta possibile sotto il vincolo di essere ortogonale alle componenti precedenti.
I vettori risultanti (essendo ciascuno una combinazione lineare delle variabili e contenente N osservazioni) costituiscono una base ortogonale.

**Componente Principale #1** $\to$ punta nella direzione della maggior varianza

**Ogni successiva Componente Principale** $\to$ ortogonale alle precedenti che punta nella direzione della maggior varianza dello spazio rimanente

In [29]:
def plot_cumulative_variance(pca, title):
    
    """
    Grafico della varianza cumulativa di tutte le PC
    
    :param (pca_object) pca: oggetto PCA
    :param (string) title: titolo del grafico
    """   

    tot_var = np.sum(pca.explained_variance_)
    ex_var = [(i / tot_var) * 100 for i in sorted(pca.explained_variance_, reverse=True)]
    cum_ex_var = np.cumsum(ex_var)

    cum_var_bar = go.Bar(
        x=list(range(1, len(cum_ex_var) + 1)), 
        y=ex_var,
        name="Varianza di ogni componente",
        marker=dict(
            color=PLOTLY_COLORS[0],
        ),
        opacity=PLOTLY_OPACITY
        )

    variance_line = go.Scatter(
        x=list(range(1, len(cum_ex_var) + 1)),
        y=cum_ex_var,
        mode='lines+markers',
        name="Varianza cumulativa",
        marker=dict(
            color=PLOTLY_COLORS[1],
        ),
        opacity=PLOTLY_OPACITY,
        line=dict(
            shape='hv',
        ))
    data = [cum_var_bar, variance_line]
    layout = go.Layout(
        autosize=True,
        title=title,
        yaxis=dict(
            title='Varianza (%)',
        ),
        xaxis=dict(
            title="Componenti principali",
            dtick=1,
            rangemode='nonnegative'
        ),
        legend=dict(
            x=0,
            y=1,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return py.iplot(fig, filename=title)

def compress_data(X_dataset, n_components, plot_comp=False, graph_title=''):
    
    """
    PCA reduction di un insieme di dati

    :param (array of arrays) X_dataset: Dataset da ridurre
    :param (int) n_components: N componenti da proiettare
    :param (bool) plot_comp: Disegno della varianza cumulativa

    :returns (pandas dataframe) X_df_reduced: Dataframe contenente il dataset ridotto
    :return (iplot) p: Visualizzare grafico, solo se plot_com vale True.
    """   

    pca = PCA(random_state=11) #random seed
    projected_data = pca.fit_transform(X_dataset)

    if plot_comp:
        p = plot_cumulative_variance(pca, graph_title)

    pca.components_ = pca.components_[:n_components]
    reduced_data = np.dot(projected_data, pca.components_.T)
    X_df_reduced = pd.DataFrame(reduced_data, columns=["PC#%d" % (x + 1) for x in range(n_components)])
    if plot_comp:
        return p, X_df_reduced
    else:
        return X_df_reduced
    

In [30]:
plot, X_df_ohc_reduced = compress_data(X_dataset=X_scaled_data,
              n_components=8,
              plot_comp=True,
              graph_title="Varianza individuale e cumulativa")
plot

In [50]:
X_df_ohc_reduced.head(4)

,PC#1,PC#2,PC#3,PC#4,PC#5,PC#6,PC#7,PC#8
0,0.674659,2.095604,3.809485,0.111267,1.151868,-1.089498,0.224776,-0.555006
1,0.732016,1.850302,-0.558395,-0.096317,2.620567,-1.278350,0.989618,-0.101235
2,-0.523563,-0.160394,-0.290459,-0.790662,-0.092465,1.432148,1.631711,0.391299
3,0.181329,-0.676779,0.040449,0.579549,0.353748,-0.006323,-0.480461,0.870860


Cercheremo ora di utilizzare uno scatter-plot per mostrare se un algoritmo di clustering applicato sulle prime due componenti principali è in grado di separare i campioni in due cluster diversi.

PCA cerca di trovare combinazioni di features che conducono alla massima separazione tra i dati; ciò significa che, se avessimo una dimensione nel nostro dataset che rimane la stessa per tutti i dati, questa non verrebbe considerata, da solo o come combinazione, tra le componenti principali. Solo le caratteristiche che variano molto da dato a dato fanno parte di esse. Di conseguenza, i punti dovrebbero apparire abbastanza distanti l'uno dall'altro sul grafico.

Il grafico potrebbe avere anche poco senso, e ciò può avvenire per due fattori principali 
1. E' presente molta varianza nel dataset, e perciò le prime due componenti non bastano a rappresentare significativamente i dati
2. L'algoritmo di clustering si focalizza su features che non sono considerate importanti da PCA

In [31]:
values = X_scaled_data
pca = PCA(n_components=2)
x = pca.fit_transform(values)

kmeans = KMeans(n_clusters=2, random_state=11)
X_clustered = kmeans.fit_predict(values)

c1_idx = np.where(X_clustered == 0)
c2_idx = np.where(X_clustered == 1)

p1 = go.Scatter(
    x=np.take(x[:,0], indices=c1_idx)[0],
    y=np.take(x[:,1], indices=c1_idx)[0],
    mode='markers',
    name="Cluster1",
    marker=dict(
        color=COLOR_PALETTE[2],
    ),
    opacity=PLOTLY_OPACITY)

p2 = go.Scatter(
    x=np.take(x[:,0], indices=c2_idx)[0],
    y=np.take(x[:,1], indices=c2_idx)[0],
    mode='markers',
    name="Cluster2",
    marker=dict(
        color=COLOR_PALETTE[5],
    ),
    opacity=PLOTLY_OPACITY)

data = [p1, p2]

layout = go.Layout(
    title='Dati clusterizzati usando le prime due componenti',
    autosize=True,
    yaxis=dict(
        title='Seconda componente',
    ),
    xaxis=dict(
        title="Prima componente",
        dtick=1,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='clusters-scatter')

### CLASSIFICAZIONE

In questa fase esploreremo diversi metodi di supervised learning e vedremo alla fine quale di questi classifica meglio i nostri dati.

Prima di iniziare, vediamo quale tipo di pre-processed data è meglio utilizzare. Siccome il nostro dataset è particolarmente piccolo, sicuramente la riduzione della dimensionalità vista poco sopra con PCA non è necessaria.

La prima cosa da fare è suddividere i dati:

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled_data,y_data, test_size=0.2, random_state=RANDOM_SEED, stratify=y_data)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_df_ohc_reduced,y_data, test_size=0.2, random_state=RANDOM_SEED, stratify=y_data)

In [49]:
def print_gridcv_scores(grid_search, n=5):
    """
    Stampa il punteggio migliore raggiunto da grid_search, con i relativi parametri

    :param (estimator) clf: oggetto con all'interno un classificatore
    :param (int) n: i migliori N punteggi
    """    
    
    t = PrettyTable()

    print("Migliori parametri su insieme di validazione:")
    indexes = np.argsort(grid_search.cv_results_['mean_test_score'])[::-1][:n]
    means = grid_search.cv_results_['mean_test_score'][indexes]
    stds = grid_search.cv_results_['std_test_score'][indexes]
    params = np.array(grid_search.cv_results_['params'])[indexes]
    
    t.field_names = ['Score'] + [f for f in params[0].keys()] 
    for mean, std, params in zip(means, stds, params):
        row=["%0.3f (+/-%0.03f)" % (mean, std * 2)] + [p for p in params.values()]
        t.add_row(row)
    print(t)
               

def param_tune_grid_cv(clf, params, X_train, y_train, cv, execution_time=False):
    """
    Function that performs a grid search over some parameters
    Funzione che effettua una gridSearch sui parametri

    :param (estimator) clf: oggetto con all'interno un classificatore
    :param (dictionary) params: dizionario di parametri su cui lavorare
    :param (array-like) X_train: dati di allenamento
    :param (array-like) y_train: true labels dei dati di allineamento
    :param (cross-validation generator) cv: determina la cross-validation splitting strategy
    """ 
    if execution_time:
      start = time.perf_counter()
    pipeline = Pipeline([('clf', clf)])
    grid_search = GridSearchCV(estimator=pipeline, 
                               param_grid=params, 
                               cv=cv, 
                               n_jobs=-1,       # Use all processors
                               scoring='f1',    # Use f1 metric for evaluation
                               return_train_score=True)
    grid_search.fit(X_train, y_train)
    if execution_time:
      end = time.perf_counter()
      return grid_search, "%.4f" % (end-start)
    return grid_search
   

def score(clfs, datasets):
    """
    Funzione che testa un classificatore su dei dati
    
    :param (array of estimator) clf: vettore di classificatori
    :param (dictionary) params: dizionario con i dati di test, passati come [(X_test, y_test)]

    """  
    scores = []
    for c, (X_test, y_test) in zip(clfs, datasets):
        scores.append(c.score(X_test, y_test))

    return scores


def hexToRGBA(hex, alpha):

    """
    Funzione che ritorna un valore rgba da valori di opacità e esadecimali
    
    :param (String) clf: valore esadecimale
    :param (float) params: valore compreso tra 0 e 1 che indica livello di opacità

    """  

    r = int(hex[1:3], 16)
    g = int(hex[3:5], 16)
    b = int(hex[5:], 16)

    if alpha:
        return "rgba(" + str(r) + ", " + str(g) + ", " + str(b) + ", " + str(alpha) + ")"
    else:
        return "rgb(" + str(r) + ", " + str(g) + ", " + str(b) + ")"


def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Genera un semplice grafico di curva di apprendimento del training e del test 
    """
    
    train_sizes, train_scores, test_scores = learning_curve(estimator, 
                                                            X, 
                                                            y, 
                                                            cv=cv, 
                                                            n_jobs=n_jobs, 
                                                            train_sizes=train_sizes, 
                                                            scoring="f1", 
                                                            random_state=RANDOM_SEED,
                                                           )
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    # Prints lower bound (mean - std) of train 
    trace1 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean - train_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[0], 0.4),
        ),
    )
    # Prints upper bound (mean + std) of train
    trace2 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean + train_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[0], 0.4),
        ),
    )
    
    # Prints mean train score line
    trace3 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean, 
        showlegend=True,
        name="Punteggio training",
        line = dict(
            color = PLOTLY_COLORS[0],
        ),
    )
    
    # Prints lower bound (mean - std) of test 
    trace4 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean - test_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[1], 0.4),
        ),
    )
        # Prints upper bound (mean + std) of test
    trace5 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean + test_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[1], 0.4),
        ),
    )

    # Prints mean test score line 
    trace6 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean, 
        showlegend=True,
        name="Punteggio test",
        line = dict(
            color = PLOTLY_COLORS[1],
        ),
    )
    
    data = [trace1, trace2, trace3, trace4, trace5, trace6]
    layout = go.Layout(
        title=title,
        autosize=True,
        yaxis=dict(
            title='Punteggio',
        ),
        xaxis=dict(
            title="#Dati training ",
        ),
        legend=dict(
            x=0.8,
            y=0,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return py.iplot(fig, filename=title)


def print_confusion_matrix(gs, X_test, y_test):

    """
    Funzione che stampa la matrice di confusione di un classificatore
    
    :param (estimator) clf: oggetto con all'interno un classificatore
    :param (array-like) X_test: dati di test
    :param (array-like) y_test: labels dei dati di test
    """  

    gs_score = gs.score(X_test, y_test)
    y_pred = gs.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    t = PrettyTable()
    t.add_row(["True malati", cm[0][0], cm[0][1]])
    t.add_row(["True sani", cm[1][0], cm[1][1]])
    t.field_names = [" ", "Predicted malati", "Predicted sani"]
    print(t)

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
    cm_df = pd.DataFrame(cm.round(3), index=["True Sick", "True Healthy"], columns=["Predicted sick", "Predicted healthy"])
    cm_df


def print_raw_score(clf, X_test, y_test):
    """
    Funzione che testa un classificatore su dei dati
    
    :param (array of estimator) clf: vettore di classificatori
    :param (array-like) X_test: dati di test
    :param (array-like) y_test: labels dei dati di test

    """  
    print("Punteggio raggiunto: %0.3f" % (score([clf], [(X_test, y_test)])[0]))


def plot_feature_importance(feature_importance, title):
    """
    Funzione che stampa l'importanza di una festure per un decision tree o un random forest classifier
    
    :param (dictionary) feature_importance: dizionario degli attributi più importanti ordinati
    :param (str) title: titolo del grafico

    """ 
    
    trace1 = go.Bar(
        x=feature_importance[:, 0],
        y=feature_importance[:, 1],
        marker = dict(color = PLOTLY_COLORS[0]),
        opacity=PLOTLY_OPACITY,
        name='Importanza feature'
    )
    data = [trace1]
    layout = go.Layout(
        title=title,
        autosize=True,
        margin=go.layout.Margin(l=50, r=100, b=150),
        xaxis=dict(
            title='feature',
            tickangle=30
        ),
        yaxis=dict(
            title='importanza feature',
            automargin=True,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return py.iplot(fig, filename=title)


def print_performances(classifiers, classifier_names, auc_scores, X_test, y_test):
  
    """
     Funzione che testa un classificatore su dei dati
    
    :param (array of estimator) clf: vettore di classificatori
    :param (array-like) classifier_names: nome del classificatore
    :param (array-like) auc-score: punteggio auc
    :param (array-like) X_test: dati di test
    :param (array-like) y_test: labels dei dati di test 

    """ 

    accs = []
    recalls = []
    precision = []
    results_table = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1", "auc"])
    for (i, clf), name, auc in zip(enumerate(classifiers), classifier_names, auc_scores):
        y_pred = clf.predict(X_test)
        row = []
        row.append(accuracy_score(y_test, y_pred))
        row.append(precision_score(y_test, y_pred))
        row.append(recall_score(y_test, y_pred))
        row.append(f1_score(y_test, y_pred))
        row.append(auc)
        row = ["%.3f" % r for r in row]
        results_table.loc[name] = row
    return results_table


In [34]:
kf = StratifiedKFold(n_splits=5, random_state=RANDOM_SEED)
clf_nb = GaussianNB()
clf_knn = KNeighborsClassifier()
clf_rf = RandomForestClassifier(random_state=RANDOM_SEED)
clf_lr = LogisticRegression(random_state=RANDOM_SEED)
clf_svm = SVC(random_state=RANDOM_SEED)

clfs = [clf_nb, clf_knn, clf_rf, clf_lr, clf_svm]
clfs_ng = [clf_nb, clf_rf]
TEST_PARAMS = {}

In [35]:
all_test_results = []
all_gss = []
times = np.zeros(2)
t = 0

for clf in clfs:
    gs_ohc_pc, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train, y_train, kf, execution_time=True)
    times[0] = times[0] + float(t)

    gs_ohc, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_pca, y_train_pca, kf, execution_time=True)
    times[1] = times[1] + float(t)

    gss = [gs_ohc_pc, gs_ohc]
    all_gss.append(gss)
    test_results = score(gss, [(X_test, y_test),
                                (X_test_pca, y_test_pca)])
    all_test_results.append(test_results)

all_test_results_ng = []
all_gss_ng = []
times_ng = np.zeros(3)

for clf in clfs_ng:
    gs_full, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train, y_train, kf, execution_time=True)
    times_ng[0] = times_ng[0] + float(t)
  
    #gs_pc, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_pc, y_train_pc, kf, execution_time=True)
    #times[1] = times[1] + float(t)
  
    gs_drop, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_pca, y_train_pca, kf, execution_time=True)
    times_ng[1] = times_ng[1] + float(t)
  
    #gs_pc_drop, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_pc_drop, y_train_pc_drop, kf, execution_time=True)
    #times[3] = times[3] + float(t)
  
    #gs_no_stalk, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train, y_train, kf, execution_time=True)
    #times_ng[2] = times_ng[2] + float(t)

    gss_ng = [gs_full, gs_drop]
    all_gss_ng.append(gss_ng)
    test_results = score(gss_ng, [(X_test, y_test),
                                  (X_test_pca, y_test_pca)
                                 ])
    all_test_results_ng.append(test_results)


In [39]:
def print_results(column_names, row_names, values):
    """
    Function that prints a table using prettytable library
    Funzione che stampa una tabella con prettytable
    
    :param (array) column_names: vettore con i nomi delle colonne
    :param (array) row_names: vettore con i nomi delle righe
    :param (matrix) values: valori della tabella

    """ 
    t = PrettyTable()
    t.field_names = column_names
    
    all_rows = []
    result_row = []

    for name, results in zip(row_names, values):
        result_row.append(name)
        for r in results:
            result_row.append("%.3f" % r)
        all_rows.append(result_row)
        result_row = []

    all_rows = sorted(all_rows, key=lambda kv: kv[1], reverse=True)
    for k in all_rows:
        t.add_row(k)
    
    print(t)


dataset_strings = [" ",
                   "Dataset con dati normalizzati",
                   "Dataset con PCA"]

#dataset_strings_ng = [" ", "full dataset",
#                      "dataset with dropped missing values"]

row_names = ["Naive Bayes", "KNN", "Random Forest", "Logistic Regression", "SVM"]
#row_names_ng = ["Naive Bayes", "Random Forest"]

print_results(dataset_strings, row_names, all_test_results)
#print_results(dataset_strings_ng, row_names_ng, all_test_results_ng)

+---------------------+-------------------------------+-----------------+
|                     | Dataset con dati normalizzati | Dataset con PCA |
+---------------------+-------------------------------+-----------------+
| Logistic Regression |             0.845             |      0.789      |
|    Random Forest    |             0.833             |      0.719      |
|         KNN         |             0.824             |      0.761      |
|         SVM         |             0.812             |      0.761      |
|     Naive Bayes     |             0.789             |      0.750      |
+---------------------+-------------------------------+-----------------+


In [40]:
print_confusion_matrix(all_gss_ng[0][np.argmin(all_test_results_ng[0])], X_test_pca, y_test_pca)

+--------------+----------------+-------------------+
|              | Predicted Sick | Predicted Healthy |
+--------------+----------------+-------------------+
|  True Sick   |       16       |         12        |
| True Healthy |       6        |         27        |
+--------------+----------------+-------------------+


In [41]:
means = np.mean(all_test_results, axis=0)
row_names = ["Tempo totale training (s)", "Punteggio medio del dataset"]
print_results(dataset_strings, row_names, [times, means])

+-----------------------------+-------------------------------+-----------------+
|                             | Dataset con dati normalizzati | Dataset con PCA |
+-----------------------------+-------------------------------+-----------------+
|  Tempo totale training (s)  |             2.295             |      1.088      |
| Punteggio medio del dataset |             0.820             |      0.756      |
+-----------------------------+-------------------------------+-----------------+


In [42]:
print("Il dataset che restituisce le performance medie migliori è:")
print("\t- " + dataset_strings[means.argmax()+1] + ", con un punteggio di " + str("%.3f" % means.max()))

Il dataset che restituisce le performance medie migliori è:
	- Dataset con dati normalizzati, con un punteggio di 0.820


### LOGISTIC REGRESSION

Il primo classificatore che andiamo ad analizzare è la logistic regression: esso usa la funzione di Sigmoid per classificare i nostri sample.

In alcuni casi, infatti, possiamo usare la regressione lineare (la quale non prova a predirre la classe ma il valore esatto partendo da un dato input **x** e calcolando un output **y**) per determinare un boundary appropriato. Tuttavia, poichè l'output è generalmente binario o discreto, esistono metodi di regressione più efficienti.
Ricordiamo che per la classificazione siamo interessati alla probabilità condizionale $P(y | x ; \theta$) dove $\theta$ descrive i parametri per il nostro modello. Quando usiamo la regressione, $\theta$ rappresenta i valori dei nostri coefficienti della regressione ($w$).

* $ P(y=0 | X;\theta) = g(w^TX) = \frac{1}{1+e^{w^TX}} $
* $ P(y=1 | X;\theta) = 1 - g(w^TX) = \frac{e^{w^TX}}{1+e^{w^TX}} $

Ma come ricaviamo i parametri? In maniera simile a come funzionano gli altri problemi di regressione, cerchiamo la Maximum Likelihood Estimation per $w$. La probabilità dei dati forniti dal modello è:

$$ L(y|x;w) = \prod_{i}(1-g(x_i;w))^{y_i}\cdot g(x_i;w)^{1-y_i} $$

Usando operazioni aritmetiche come usare la funzione <code>log</code> otteniamo:

$$ LL(y|x;w) = \sum_{i=1}^{N}y_iln(1-g(x_i;w))+(1-y_i)ln(g(x_i;w)) = \sum_{i=1}^{N}y_iln(1-g(x_i;w))-y_iln(g(x_i;w))+ln(g(x_i;w)) = $$

$$ = \sum_{i=1}^{N}y_iln\left(\frac{1-g(x_i;w)}{g(x_i;w)}\right) + ln(g(x_i;w)) = \sum_{i=1}^{N}y_iw^Tx_i - ln(1+e^{w^Tx_i}) = \ell(w)$$

$$ \frac{\partial \ell(w)}{\partial w_j} = \sum_{i=1}^{N}x_i^j[y_i - (1 - g(x_i;w))] = \sum_{i=1}^{N}x_i^j[y_i - P(y_i=1|x_i;w)] $$

In questo modo otteniamo una funzione concava, che ci permette di arrivare a una soluzione.

Questo modello, rispetto alla regressione lineare, può modellare meglio la zona compresa tra 0 e 1. Per conoscere i pesi, bisogna calcolare la MLE (Maximum Likelihood Estimation, ossia scegliere un valore che massimizzi la probabilità dei dati osservati) ed applicare l'algoritmo di gradient descent fino alla convergenza del valore di accuratezza.

Essi sono:
* **liblinear**: Solver, migliore per dataset più piccoli.
* **C**: "forza di regolarizzazione", valore compreso tra 0,01 e 100. Più è piccolo il valore, maggiore è la regolarizzazione
* **penalty**: penalità "l1" e "l2" per la regolarizzazione, che sono definite come:
    * l1, che penalizza ogni errore allo stesso modo $ S = \sum_{i=1}^{n}| y_i - f(x_i) | $
    * l2, che penalizza maggiormente valori più grandi $ S = \sum_{i=1}^{n}(y_i - f(x_i))^2 $

Dove $y_i$ è la vera label mentre $f(x_i)$ è quella assegnata.    

In [43]:
clf_lr = LogisticRegression(random_state=RANDOM_SEED)
gs_pc_lr = param_tune_grid_cv(clf_lr, LOGISTIC_REGRESSION_PARAMS, X_train, y_train, kf)
print_gridcv_scores(gs_pc_lr, n=5)

Migliori parametri su insieme di validazione:
+------------------+--------+--------------+-------------+
|      Score       | clf__C | clf__penalty | clf__solver |
+------------------+--------+--------------+-------------+
| 0.868 (+/-0.032) |  0.1   |      l2      |    lbfgs    |
| 0.868 (+/-0.032) |  0.1   |      l2      |  newton-cg  |
| 0.867 (+/-0.031) |  0.1   |      l2      |  liblinear  |
| 0.865 (+/-0.014) | 0.001  |      l2      |  liblinear  |
| 0.863 (+/-0.010) |  0.01  |      l2      |  liblinear  |
+------------------+--------+--------------+-------------+


In [46]:
plot_learning_curve(gs_pc_lr.best_estimator_, "Curva di apprendimento di Logistic Regression", 
                    X_train,
                    y_train,
                    cv=5,
                   )

### SUPPORT VECTOR MACHINE

Una Support Vector Machine (SVM) è un classificatore discriminativo definito formalmente da un iperpiano di separazione.
In altre parole, dati dei labeled sample (quindi un supervised learning), l'output dell'algoritmo genera un iperpiano ottimale che classifica poi i nuovi esempi. 

Tra gli algoritmi di machine learning, SVM è molto popolare per la sua potenza soprattuto nella casistica di dover risolvere un problema di classificazione binaria (come quello di questa tesina), sia in presenza di dati lineari che non lineari. 

Caratteristica di SVM è la presenza dell'utilizzo di funzioni kernel: esse vengono utilizzate per identificare elementi in uno spazio senza utilizzare le coordinate ma invece calcolando il prodotto interno delle immagini di tutte le coppie di dati nello spazio della funzione. Questa operazione risulta spesso computazionalmente più efficiente del calcolo delle coordinate e viene chiamata *kernel trick*. 

Il Support Vector Machine apprende i boundaries tra dati appartenenti a due diverse classi proiettandoli in uno spazio multidimensionale allo scopo di trovare l'iperpiano che massimizza i margini tra le due classi di dati: ma qual è l'iperpiano ottimale da scegliere, essendocene un numero infinito? 
Per calcolarlo, l'algoritmo ne genera una serie e prova a separare le istanze tramite la massimizzazione dei margini tra i due punti più vicini delle due classi.
I dati vengono separati tramite un boundary e i punti che si trovano su di esso sono detti *vettori di supporto*: non vengono utilizzati quindi tutti i dati categorizzati ma solo quelli sui margini.

Dobbiamo quindi trovare l'iperpiano che massimizza i margini. Il caso più semplice è quando i dati sono linearmente separabili. 
Pur essendo particolarmente efficace in casi di classificazione binaria, questo algoritmo si presta anche per la classificazione di classi multiple, con indici di robustezza e accuratezza inferiori. 

In uno spazio a due dimensioni, tale iperpiano è definito come una retta che divide il piano in due parti, ognuna che specifica una delle due classi.

In questo caso abbiamo codeste specifiche:
* **linear**: la più semplice delle SVM, trova l'iperpiano che separa nel migliore dei modi i nostri dati di training
* **C**: il parametro C di SVM dice all'ottimizzatore una misura su quanto è importante evitare classificazioni errate. Più è alto il valore di C, più sarà piccolo il margine della retta SE essa con tale margine è in grado di classificare meglio i nostri dati.
* **rbf**: questo parametro indica che stiamo usando una Radial Basis Function kernel per effettuare il prodotto scalare
    * **gamma**: definisce fino a che punto il valore di un singolo elemento possa influire; se il valore di gamma è basso, significa molta influenza, con un valore alto, poca influenza. 

In [47]:
clf_svm = SVC(probability=True, random_state=RANDOM_SEED)
gs_pc_svm = param_tune_grid_cv(clf_svm, SVM_PARAMS, X_train, y_train, kf)
print_gridcv_scores(gs_pc_svm, n=5)

Migliori parametri su insieme di validazione:


Exception: Row has incorrect number of values, (actual) 3!=4 (expected)

In [30]:
plot_learning_curve(gs_pc_svm.best_estimator_, "Learning curve of SVM", 
                    X_train,
                    y_train,
                    cv=5)

In [32]:
print_confusion_matrix(gs_pc_svm, X_test, y_test)

+--------------+----------------+-------------------+
|              | Predicted Sick | Predicted Healthy |
+--------------+----------------+-------------------+
|  True Sick   |       20       |         8         |
| True Healthy |       5        |         28        |
+--------------+----------------+-------------------+


### Naïve Bayes Classifier

Il Naïve Bayes Classifier è un algoritmo predittivo di tipo statistico. Uno dei fondamenti della statistica è la stima della probabilità che un evento accada oppure no. Le rilevazioni statistiche possono essere totali (prendendo, per esempio, in esame un'intera popolazione: difficili e costose) oppure campionarie. 

Quest'ultima modalità è chiaramente meno costosa ma porta con sè diverse problematiche. Infatti, quando si sceglie un campione, bisogna fare in modo che esso rappresenti, in piccolo, la popolazione totale: solo in questo modo possiamo trarre conclusioni che poi possono essere probabilisticamente estese.

Si introduce quindi il concetto di probabilità: dato un fenomeno aleatorio, con un insieme di risultati tutti egualmente possibili, la probabilità di un evento è definita dal rapporto tra il numero di risultati favorevoli all'evento stesso e il numero di risultati possibili.

Il classificatore preso in oggetto si basa sul teorema di Bayes, che afferma che $ P(A|B) = \frac{P(B|A)P(A)}{P(B)} $ dove:
* $P(A)$ è il prior, il grado iniziale di "fede" in A
* $P(A|B)$ è il posteriore, il grado di credenza che rappresenta B
* $P(B|A)$ è la probabilità, il grado di credenza di B, assunto che A sia vero

Utilizzando il teorema di Bayes, possiamo quindi trovare la probabilità che A si verifichi, assumendo che B si è verificato.
Ergo, in questo caso A è l'ipotesi e B la tesi.
L'assunzione che facciamo noi è che i predittori/gli attributi siano indipendenti. Perciò la presenza di una particolare caratteristica non influenza l'altra. Per questo è chiamato "ingenuo".

In [34]:
clf_nb = GaussianNB()
gs_pc_nb = param_tune_grid_cv(clf_nb, TEST_PARAMS, X_train, y_train, kf)
print_gridcv_scores(gs_pc_nb, n=5)

print_confusion_matrix(gs_pc_nb, X_test, y_test)

Best grid scores on validation set:
+------------------+
|      Score       |
+------------------+
| 0.843 (+/-0.049) |
+------------------+
+--------------+----------------+-------------------+
|              | Predicted Sick | Predicted Healthy |
+--------------+----------------+-------------------+
|  True Sick   |       18       |         10        |
| True Healthy |       5        |         28        |
+--------------+----------------+-------------------+


In [36]:
plot_learning_curve(clf_nb, "Learning curve of GaussianNB", 
                    X_train, 
                    y_train, 
                    cv=5)

### RANDOM FOREST CLASSIFIER

E' un metodo basato sugli alberi, derivato dall'"impacchettare" gli alberi decisionali, al quale viene aggiunto un piccolo trucco che de-correla gli alberi, al fine di ridurre ulteriormente la varianza del dataset.

Come nell'impacchettamento, costruiamo un numero di alberi decisionali sui training samples. Ma, quando li costruiamo, ogni volta che viene considerata la divisione in un albero, viene scelta una selezione casuale di *m* predittori come candidati separati dal set di *p* predittori.
La divisione è autorizzata a selezionare solo uno di questi *m* predittori, che tipicamente sono $ m \approx \sqrt{p} $

In [37]:
clf_pc_rf = RandomForestClassifier(random_state=RANDOM_SEED)
gs_pc_rf = param_tune_grid_cv(clf_pc_rf, RANDOM_FOREST_PARAMS, X_train, y_train, kf)
print_gridcv_scores(gs_pc_rf, n = 5)

Best grid scores on validation set:
+------------------+----------------+----------------+-------------------+-------------------+
|      Score       | clf__criterion | clf__max_depth | clf__max_features | clf__n_estimators |
+------------------+----------------+----------------+-------------------+-------------------+
| 0.853 (+/-0.027) |      gini      |       50       |        sqrt       |        100        |
| 0.853 (+/-0.027) |      gini      |       75       |        sqrt       |        100        |
| 0.853 (+/-0.027) |      gini      |      100       |        log2       |        100        |
| 0.853 (+/-0.027) |      gini      |      100       |        sqrt       |        100        |
| 0.853 (+/-0.027) |      gini      |       75       |        log2       |        100        |
+------------------+----------------+----------------+-------------------+-------------------+


In [38]:
print_confusion_matrix(gs_pc_rf, X_test, y_test)

+--------------+----------------+-------------------+
|              | Predicted Sick | Predicted Healthy |
+--------------+----------------+-------------------+
|  True Sick   |       20       |         8         |
| True Healthy |       5        |         28        |
+--------------+----------------+-------------------+


In [39]:
plot_learning_curve(gs_pc_rf.best_estimator_, "Learning curve of Random Forest Classifier", 
                    X_train,
                    y_train,
                    cv=5)

In [44]:
feature_importance = np.array(sorted(zip(X_data.columns, 
                              gs_pc_rf.best_estimator_.named_steps['clf'].feature_importances_),
                              key=lambda x: x[1], reverse=True))
plot_feature_importance(feature_importance, "Feature importance in the random forest")

### K-NEAREST NEIGHBORS CLASSIFIER

il KNN è un tipo di apprendimento basato sull'istanza o apprendimento pigro, in cui la funzione viene solo approssimata localmente e tutti i calcoli vengono posticipati fino alla classificazione. L'algoritmo KNN è tra i più semplici di tutti gli algoritmi di apprendimento automatico.

La fase di addestramento dell'algoritmo consiste solo nel memorizzare i vettori di caratteristiche e le etichette di classe dei campioni di addestramento. Nella fase di classificazione, K è una costante definita dall'utente e un vettore senza etichetta (una query o un punto di prova) viene classificato assegnando l'etichetta più frequente tra i campioni di addestramento k più vicini a quel punto di ricerca.

I parametri del cross validation sono:
* **n_neighbors**: il numero di samples "vicini" da analizzare
* **weights**: indica la funzione weight da applicare nella predizione
    * **uniform**: tutti i punti "nel vicinato" sono pesati in maniera uguale
    * **distance**: i punti sono pesati per l'inverso della loro distanza. Perciò, i punti più vicini avranno più peso di quelli più distanti
* **p**: parametro di potenza per la metrica di Minkowski
    * $p=1$ si usa l1
    * $p=2$ si usa l2
    * $p>2$ è usata la minkowsi_distance (l_p)

In [69]:
clf_knn = KNeighborsClassifier()
gs_knn = param_tune_grid_cv(clf_knn, KNN_PARAMS, X_train, y_train, kf)
print_gridcv_scores(gs_knn, n=5)

Best grid scores on validation set:
+------------------+------------------+--------+--------------+
|      Score       | clf__n_neighbors | clf__p | clf__weights |
+------------------+------------------+--------+--------------+
| 0.864 (+/-0.014) |        15        |   2    |   distance   |
| 0.864 (+/-0.063) |        5         |   2    |   uniform    |
| 0.864 (+/-0.063) |        5         |   2    |   distance   |
| 0.863 (+/-0.070) |        5         |   1    |   uniform    |
| 0.860 (+/-0.066) |        5         |   1    |   distance   |
+------------------+------------------+--------+--------------+


In [70]:
print_confusion_matrix(gs_knn, X_train, y_train)

+--------------+----------------+-------------------+
|              | Predicted Sick | Predicted Healthy |
+--------------+----------------+-------------------+
|  True Sick   |      110       |         0         |
| True Healthy |       0        |        132        |
+--------------+----------------+-------------------+


In [71]:
plot_learning_curve(gs_knn.best_estimator_, "Learning curve of k-NN Classifier", 
                    X_train,
                    y_train,
                    cv=5)

### ROC CURVE

Arrivati a questo punto dobbiamo comparare le performance dei diversi classificatori.
Andiamo quindi a tracciare la ROC curve e la Area Under Curve per tutti i nostri modelli. La ROC curve è tracciata con TPR rispetto a FPR dove TPR è sull'asse y e FPR è sull'asse x. Nello specifico, questi parametri sono:
* $ TRP/Recall/Sensitivity = \frac{TP}{TP+FN}$
* $FPR = \frac{FP}{TN+FP}$

Un modello eccellente ha l'AUC vicino all'1 che significa che ha una buona misura di separabilità. Un modello scadente ha l'AUC vicino allo 0, il che significa che ha la peggiore misura di separabilità.

In [72]:
def plot_roc_curve(classifiers, legend, title, X_test, y_test):
    t1 = go.Scatter(
        x=[0, 1], 
        y=[0, 1], 
        showlegend=False,
        mode="lines",
        name="",
        line = dict(
            color = COLOR_PALETTE[0],
        ),
    )
    
    data = [t1]
    aucs = []
    for clf, string, c in zip(classifiers, legend, COLOR_PALETTE[1:]):
        y_test_roc = np.array([([0, 1] if y else [1, 0]) for y in y_test])
        y_score = clf.predict_proba(X_test)
        
        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(2):
            fpr[i], tpr[i], _ = roc_curve(y_test_roc[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test_roc.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        aucs.append(roc_auc['micro'])

        trace = go.Scatter(
            x=fpr['micro'], 
            y=tpr['micro'], 
            showlegend=True,
            mode="lines",
            name=string + " (area = %0.2f)" % roc_auc['micro'],
            hoverlabel = dict(
                namelength=30
            ),
            line = dict(
                color = c,
            ),
        )
        data.append(trace)

    layout = go.Layout(
        title=title,
        autosize=False,
        width=550,
        height=550,
        yaxis=dict(
            title='True Positive Rate',
        ),
        xaxis=dict(
            title="False Positive Rate",
        ),
        legend=dict(
            x=0.4,
            y=0.06,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return aucs, iplot(fig, filename=title)


In [73]:
classifiers = [gs_pc_lr, gs_pc_svm, gs_pc_nb, gs_pc_rf, gs_knn]
classifier_names = ["Logistic Regression", "SVM", "GaussianNB", "Random Forest", "KNN"]
auc_scores, roc_plot = plot_roc_curve(classifiers, classifier_names, "ROC curve", X_test, y_test)
roc_plot

Ora esaminiamo alcuni parametri aggiuntivi che valutano la bontà dei nostri modelli:
1. Accuratezza
2. Precisione 
    * $ P = \frac{TP}{TP+FP} $
3. Recall
    * $ R = \frac{TP}{TP+FN} $
4. F1 (media ponderata della precisione e recall) 
    * $ F1 = 2 * \frac{P*R}{P+R} $
5. Auc, area under the ROC curve

In [75]:
print_performances(classifiers, classifier_names, auc_scores, X_test, y_test)

,accuracy,precision,recall,f1,auc
Logistic Regression,0.836,0.811,0.909,0.857,0.908
SVM,0.787,0.778,0.848,0.812,0.880
GaussianNB,0.754,0.737,0.848,0.789,0.874
Random Forest,0.787,0.778,0.848,0.812,0.893
KNN,0.820,0.789,0.909,0.845,0.902


commenti sul grafico sopra

### CONCLUSIONI

Dalla letteratura medica è bene segnalare che solitamente, ad eccezione di qualche caso specifico, questo tipo di problemi diminuisce drasticamente nel caso in cui si prendano delle precauzioni quali ad esempio:

* Smettere di fumare
* Effettuare controlli periodici sulla condizione di salute di parametri come ipertensione, colesterolo alto e diabete
* Fare esercizio almeno 30 minuti al giorno 
* Seguire una dieta sana e pover di sale e grassi saturi
* Ridurre e gestire lo stress
* Praticare una buona igiene

